In [1]:
print("test")

test


In [2]:
%pwd

'c:\\Users\\HP\\Desktop\\doss2\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Desktop\\doss2'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:
#Extract Data From the PDF FILE

def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()  
                          
    return documents                                          

In [7]:
!pip install langchain
!pip install pypdf2


In [7]:
extracted_data = load_pdf_file(data='C:/Users/HP/Desktop/doss2/Data')

In [8]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks:", len(text_chunks))

Length of Text Chunks: 1334


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
#Dowload the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\HP\AppData\Local\Temp\ipykernel_16640\2040198703.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\HP\anaconda3\envs\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
query_result = embeddings.embed_query("hello world")
print("Length",len(query_result))

Length 384


In [17]:
pip install pinecone

  Using cached pinecone-6.0.2-py3-none-any.whl.metadata (9.0 kB)
Using cached pinecone-6.0.2-py3-none-any.whl (421 kB)
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [16]:
from dotenv import load_dotenv
load_dotenv()


True

In [17]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')


In [18]:
import os
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')


In [19]:
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')


In [27]:
pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "chatbot111"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "chatbot111",
    "metric": "cosine",
    "host": "chatbot111-5l5w2pl.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [20]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [60]:
from langchain_pinecone import PineconeVectorStore

index_name = "chatbot111"

docssearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [61]:
pip install langchain

In [62]:
pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [63]:
pip install langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [32]:
from langchain_pinecone import PineconeVectorStore
index_name = "chatbot111"
docssearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [33]:
# load existing index
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embedding into your pinecone index
docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embeddings
)

In [39]:
docsearch

In [36]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":30})

In [37]:
retrieved_docs=retriever.invoke("c'est quoi la reglementation?")

In [38]:
retrieved_docs

[Document(id='fc6b7c95-8093-48ba-9d79-ffd10b0c52c8', metadata={'author': 'AMMERMAN Zachary', 'creationdate': '2018-10-04T17:52:28-04:00', 'creator': 'Microsoft® Word 2016', 'moddate': '2018-10-04T17:52:28-04:00', 'page': 3.0, 'page_label': '4', 'producer': 'Microsoft® Word 2016', 'source': 'C:\\Users\\HP\\Desktop\\doss2\\Data\\exporter_des_denrees_alimentaires_aux_etats-unis_-_dispositions_reglementaires_sanitaires_et_douanieres2.pdf', 'total_pages': 5.0}, page_content='Elles font l’objet d’une réglementation spécifique (l’usine de production doit avoir mis \nau point un plan HACCP spécifique) visant à garantir leur bonne conservation. De plus,'),
 Document(id='00a3820b-6691-49f7-902b-185c30242c1c', metadata={'author': 'mBoufdaila', 'creationdate': '2012-12-19T16:12:32+00:00', 'creator': 'Microsoft® Office Word 2007', 'moddate': '2012-12-19T16:12:32+00:00', 'page': 14.0, 'page_label': '15', 'producer': 'Microsoft® Office Word 2007', 'source': 'C:\\Users\\HP\\Desktop\\doss2\\Data\\livra

In [73]:
pip install groq

Note: you may need to restart the kernel to use updated packages.


In [74]:
pip install langchain-groq

  Using cached langchain_groq-0.3.2-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_groq-0.3.2-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


In [48]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq

# Charger les variables d’environnement
load_dotenv()

llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama3-8b-8192"
)


In [52]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
          "you are an assistant for question answering tasks. " 
          "Use the following pieces of retrieved context to answer"
          "the question. If you don't know the answer, say that you "
          "don't know.Use three sentences maximun and keep the "
          "answer concise. "
          "\n\n"
          "{context}"

)
prompt= ChatPromptTemplate.from_messages(
        [
            ("system",system_prompt),
            ("human","{input}"),

        ]
)

In [53]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response=rag_chain.invoke({"input":"c'est quoi les normes iso pour la securite sanitaire?"})
print(response["answer"])  

The ISO 22000 standard is a international standard for food safety management systems. It provides a framework for ensuring the safety and quality of food products by establishing a system of controls and procedures to prevent foodborne illnesses.

The standard is designed to be used by organizations in the food industry, including manufacturers, processors, distributors, and retailers, as well as by governments and regulatory bodies. It helps to identify and manage food safety risks, and to ensure compliance with food safety regulations and laws.

The ISO 22000 standard is based on the Hazard Analysis and Critical Control Points (HACCP) system, which is a widely accepted approach to food safety management. It provides a structured approach to identifying and controlling hazards, and to ensuring that food products are safe for consumption.

The standard consists of a set of requirements and guidelines that organizations must follow to ensure the safety and quality of their food product